## Setup

In [2]:
!pip install --no-cache-dir /opt/torch2trt

Processing /opt/torch2trt
  Preparing metadata (setup.py) ... done
  Created wheel for torch2trt: filename=torch2trt-0.5.0-py3-none-any.whl size=53583 sha256=0bbcf3ca5972ba989647c88e07c37f214e43bd6524d8d82b362a39e9fd8dd152
  Stored in directory: /tmp/pip-ephem-wheel-cache-43z6j2l_/wheels/2b/d9/e6/a0faeee5b34ac625999e76ad15d3b02c454ff49960cd518ec4
Successfully built torch2trt


In [1]:
import torch

torch.cuda.is_available()

True

In [14]:
!ros2 topic hz /camera_ids/image_color -w 20

1726850981.362370 [42]       ros2: config: //CycloneDDS/Domain/General: 'NetworkInterfaceAddress': deprecated element (/root/.ros/cyclonedds.xml line 5)
average rate: 1.334
	min: 0.179s max: 1.321s std dev: 0.57088s window: 2
average rate: 3.078
	min: 0.103s max: 1.321s std dev: 0.38085s window: 8
average rate: 4.874
	min: 0.092s max: 1.321s std dev: 0.27650s window: 18
average rate: 7.148
	min: 0.092s max: 0.303s std dev: 0.05895s window: 20
average rate: 7.694
	min: 0.097s max: 0.206s std dev: 0.04455s window: 20
average rate: 8.020
	min: 0.096s max: 0.244s std dev: 0.04435s window: 20
average rate: 7.407
	min: 0.096s max: 0.259s std dev: 0.05300s window: 20
average rate: 6.896
	min: 0.096s max: 0.259s std dev: 0.05535s window: 20
average rate: 7.138
	min: 0.097s max: 0.259s std dev: 0.05242s window: 20
average rate: 7.695
	min: 0.097s max: 0.297s std dev: 0.05546s window: 20
average rate: 8.409
	min: 0.074s max: 0.297s std dev: 0.05182s window: 20
average rate: 7.765
	min: 0.074s ma

In [3]:
# Training is performed on (3, 518, 518) images
# Evaluation in paper is performed on (3, 518, <some_multiple_of_14>) or (3, <some_multiple_of_14>, 518) with 518 for the smaller dimension and while keeping the original aspect ratio
# Paper states that inference works on other image scales as well (basically as long as dims are multiples of 14?)
shape_original = (1, 3, 1920, 2556)
min_size_resized = 350
base = 14
name_model = "depth_anything_v2_metric_hypersim_vitb"


import nimbro_depthanything.transforms as transforms

shape_input = transforms.resize_shape_to_multiple_of_base(shape_original, min_size_resized=min_size_resized, base=base)
shape_input

(1, 3, 350, 462)

In [4]:
import nimbro_depthanything.scripts.export_model as export_model

export_model.export_model(name_model, shape_original=shape_original, min_size_resized=min_size_resized, base=base)

xFormers not available
xFormers not available


Exporting model ...


/root/colcon_ws/src/nimbro_depthanything/nimbro_depthanything/models/dinov2_layers/patch_embed.py:73: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert H % patch_H == 0, f"Input image height {H} is not a multiple of patch height {patch_H}"
/root/colcon_ws/src/nimbro_depthanything/nimbro_depthanything/models/dinov2_layers/patch_embed.py:74: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert W % patch_W == 0, f"Input image width {W} is not a multiple of patch width: {patch_W}"
/root/colcon_ws/src/nimbro_depthanything/nimbro_depthanything/models/dino

Model exported to /root/colcon_ws/src/nimbro_depthanything/resources/models_onnx/depth_anything_v2_metric_hypersim_vitb.onnx


In [5]:
import nimbro_depthanything.scripts.build_engine as build_engine

build_engine.build_engine(name_model, limit_memory_workspace=1 << 32)

Building tensorrt engine ...
[09/20/2024-17:45:26] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 645, GPU 4861 (MiB)
[09/20/2024-17:45:29] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +955, GPU +938, now: CPU 1647, GPU 5795 (MiB)
[09/20/2024-17:45:30] [TRT] [I] ----------------------------------------------------------------
[09/20/2024-17:45:30] [TRT] [I] Input filename:   /root/colcon_ws/src/nimbro_depthanything/resources/models_onnx/depth_anything_v2_metric_hypersim_vitb.onnx
[09/20/2024-17:45:30] [TRT] [I] ONNX IR version:  0.0.8
[09/20/2024-17:45:30] [TRT] [I] Opset version:    17
[09/20/2024-17:45:30] [TRT] [I] Producer name:    pytorch
[09/20/2024-17:45:30] [TRT] [I] Producer version: 2.3.0
[09/20/2024-17:45:30] [TRT] [I] Domain:           
[09/20/2024-17:45:30] [TRT] [I] Model version:    0
[09/20/2024-17:45:30] [TRT] [I] Doc string:       
[09/20/2024-17:45:30] [TRT] [I] ----------------------------------------------------------------
[09/20/202

## Run node

In [5]:
import IPython.display as display
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorrt as trt
import torch
from torch2trt import TRTModule
import torchvision.transforms.v2 as tv_transforms


from cv_bridge import CvBridge
from message_filters import ApproximateTimeSynchronizer, Cache, Subscriber as SubscriberFilter
from rclpy.callback_groups import MutuallyExclusiveCallbackGroup, ReentrantCallbackGroup
from rclpy.duration import Duration
from rclpy.time import Time
from rclpy.node import Node
from rclpy.qos import HistoryPolicy, ReliabilityPolicy, QoSProfile
from rcl_interfaces.msg import FloatingPointRange, IntegerRange, ParameterDescriptor, ParameterType
from sensor_msgs.msg import CameraInfo, Image, PointCloud2, PointField
from std_msgs.msg import Header
from tf2_ros import TransformBroadcaster
from tf2_ros.buffer import Buffer
from tf2_ros.transform_listener import TransformListener

import nimbro_utils.compat.point_cloud2 as point_cloud2
from nimbro_utils.parameter_handler import ParameterHandler
from nimbro_utils.tf_oracle import TFOracle

import time

import nimbro_depthanything.transforms as transforms
import nimbro_depthanything.config as config


class NodeDepthAnything(Node):
    def __init__(
        self,
        topic_image="/camera_ids/image_color",
        topic_inferred_depth="/camera_ids/inferred/depth/image",
        shape_original=(1, 3, 1920, 2556),
        min_size_resized=350,
        base=14,
        name_model="depth_anything_v2_metric_hypersim_vitb",
        max_depth=20,
    ):
        super().__init__(node_name="depth_anything")

        self.bridge_cv = None
        self.device = None
        self.handler_parameters = None
        self.max_depth = max_depth
        self.name_model = name_model
        self.shape_original = shape_original
        self.min_size_resized = min_size_resized
        self.base = base
        self.profile_qos = None
        self.subscriber_image = None
        self.topic_image = topic_image
        self.topic_inferred_depth = topic_inferred_depth

        self._init()

    def _init(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.bridge_cv = CvBridge()
        self.profile_qos = QoSProfile(reliability=ReliabilityPolicy.BEST_EFFORT, history=HistoryPolicy.KEEP_LAST, depth=1)
        self.handler_parameters = ParameterHandler(self, verbose=False)

        # self._init_parameters()

        # self._init_tf_oracle()
        # self._del_publishers()

        # self._del_services()
        # self._init_services()
        # self._del_subscribers()

        path_engine = Path(config._PATH_DIR_ENGINES) / f"{self.name_model}.engine"
        logger = trt.Logger(trt.Logger.INFO)
        runtime = trt.Runtime(logger)
        with open(path_engine, "rb") as file_engine:
            engine_serialized = file_engine.read()
        engine = runtime.deserialize_cuda_engine(engine_serialized)
        engine.get_tensor_shape("input")
        self.model = TRTModule(
            engine=engine,
            input_names=["input"],
            output_names=["output"],
        )
        self.model = self.model.to(self.device)

        self.transform = tv_transforms.Compose(
            [
                tv_transforms.PILToTensor(),
                transforms.ResizeToMultipleOfBase(min_size_resized=self.min_size_resized, base=self.base, mode_interpolation=transforms.MODES_INTERPOLATION.BICUBIC, use_antialiasing=False),
                tv_transforms.ToDtype(dtype=torch.float32, scale=True),
                tv_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        )

        # self.i = 0
        # stream = torch.cuda.Stream()

        self.get_logger().info(f"Initialized model")

        self._init_subscribers()
        self._init_publishers()

    def _init_subscribers(self):
        self.subscriber_image = self.create_subscription(Image, self.topic_image, self.infer_depth, qos_profile=self.profile_qos, callback_group=MutuallyExclusiveCallbackGroup())

    def _init_publishers(self):
        self.publisher_depth = self.create_publisher(msg_type=Image, topic=self.topic_inferred_depth, qos_profile=self.profile_qos, callback_group=ReentrantCallbackGroup())

    def publish_image(self, image, name_frame, stamp):
        header = Header(stamp=stamp, frame_id=name_frame)
        message = self.bridge_cv.cv2_to_imgmsg(image, header=header, encoding="mono16")

        self.publisher_depth.publish(message)

    def visualize_depth_image(self, image_depth, image_rgb, time_needed):
        din_a4 = np.array([210, 297]) / 25.4
        din_a4_landscape = din_a4[::-1]
        fig = plt.figure(figsize=din_a4_landscape)

        def visualize_image(image):
            ax = plt.gca()
            ax.set_axis_off()
            ax.imshow(image, cmap="turbo" if image.shape[2] == 1 else None, vmin=0)

        fig.add_subplot(1, 2, 1)
        visualize_image(image_rgb)

        fig.add_subplot(1, 2, 2)
        visualize_image(image_depth)

        plt.tight_layout()
        plt.title(f"Inference time: {time_needed:.4f}, Hz {1 / time_needed}")
        # plt.savefig(Path(".") / "gif" / f"image_{self.i}")
        # self.i += 1
        plt.show()

    @torch.inference_mode()
    def forward_input_through_model(self, input):
        output = self.model(input)
        output *= 1000

        return output

    @torch.inference_mode()
    def infer_depth(self, message_image):
        s = time.time()
        image = self.bridge_cv.imgmsg_to_cv2(message_image, desired_encoding="passthrough")
        image = torch.from_numpy(image).permute((2, 0, 1)).to(self.device)

        input = image[None, ...]
        input = self.transform(input)

        output = self.forward_input_through_model(input)
        output = output[:, None, :, :]
        output = torch.nn.functional.interpolate(output, (self.shape_original[2], self.shape_original[3]), mode="bilinear", align_corners=True)
        output = output[0]
        output = output.cpu().numpy()

        self.get_logger().info(f"Mean depth: {output.mean()}")

        self.publish_image(output.transpose((1, 2, 0)).astype(np.uint16), name_frame=message_image.header.frame_id, stamp=message_image.header.stamp)

        e = time.time()
        time_needed = e - s

        # display.clear_output(wait=True)
        # output = output / output.max()
        self.visualize_depth_image(output.transpose((1, 2, 0)), image.permute((1, 2, 0)).cpu().numpy(), time_needed)

In [6]:
import nimbro_utils.node as utils_node
import rclpy

try:
    rclpy.shutdown()
except:
    pass

utils_node.start_and_spin_node(NodeDepthAnything)

1726849086.699686 [42] pt_main_th: config: //CycloneDDS/Domain/General: 'NetworkInterfaceAddress': deprecated element (/root/.ros/cyclonedds.xml line 5)


[09/20/2024-18:18:06] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[09/20/2024-18:18:06] [TRT] [I] Loaded engine size: 372 MiB
[09/20/2024-18:18:07] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +125, now: CPU 0, GPU 993 (MiB)
[09/20/2024-18:18:07] [TRT] [W] Using default stream in enqueueV3() may lead to performance issues due to additional calls to cudaStreamSynchronize() by TensorRT to ensure correct synchronization. Please use non-default stream instead.


[INFO] [1726849087.261936992] [depth_anything]: Initialized model
[INFO] [1726849087.540854684] [depth_anything]: Mean depth: 3067.193115234375
[INFO] [1726849087.587198727] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849087.858477801] [depth_anything]: Mean depth: 2893.926025390625
[INFO] [1726849087.901286127] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849088.150300821] [depth_anything]: Mean depth: 2978.096923828125
[INFO] [1726849088.190198536] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849088.435660617] [depth_anything]: Mean depth: 2984.9013671875
[INFO] [1726849088.474736485] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849088.709021191] [depth_anything]: Mean depth: 2999.3193359375
[INFO] [1726849088.739935386] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849089.004163090] [depth_anything]: Mean depth: 3048.186767578125
[INFO] [1726849089.050158100] [depth_anything]: Shape: (1, 1920, 2556)
[INFO] [1726849089.348222131] [depth_anythin

Node interrupted
